In [1]:
import datasets
from datasets import load_dataset

In [6]:
datasets_to_parse = [
    ("coref-data/preco_indiscrim", "default"),
    ("coref-data/litbank_indiscrim", "split_0"),
    ("coref-data/arrau_indiscrim", "default"),
    ("coref-data/phrase_detectives_indiscrim", "default"),
    ("coref-data/mmc_indiscrim", "mmc_en"),
]

# ("coref-data/gum_indiscrim", "ontogum"), # just constituency
# ("coref-data/gum_indiscrim", "original"), # just constituency

# "misc": { "parse_tree"
# { "deprel": "prep", "head": 2, "id": 3, "text": "of", "upos": "IN", "xpos": "IN" }

In [31]:
dataset_name, dataset_config = datasets_to_parse[0]
dataset = load_dataset(dataset_name, dataset_config)

In [30]:
import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse,constituency', tokenize_pretokenized=True)

2024-02-12 21:31:43 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-02-12 21:31:44 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |

2024-02-12 21:31:44 INFO: Using device: cpu
2024-02-12 21:31:44 INFO: Loading: tokenize
2024-02-12 21:31:44 INFO: Loading: mwt
2024-02-12 21:31:44 INFO: Loading: pos
2024-02-12 21:31:44 INFO: Loading: lemma
2024-02-12 21:31:44 INFO: Loading: constituency
2024-02-12 21:31:44 INFO: Loading: depparse
2024-02-12 21:31:44 INFO: Done loading processors!


In [34]:
def add_parse_to_example(example):
    sentences = example["sentences"]
    words = [[t["text"] for t in s["tokens"]] for s in sentences]
    doc = nlp(words)
    
    for stanza_sentence, sentence in zip(doc.sentences, sentences):
        sentence["misc"] = {"parse_tree": str(stanza_sentence.constituency)}
        for stanza_word, token in zip(stanza_sentence.words, sentence["tokens"]):
            assert stanza_word.text == token["text"] and stanza_word.id == token["id"]
            word_dict = stanza_word.to_dict()
            for key, value in word_dict.items():
                token[key] = value

In [33]:
from tqdm import tqdm

validation_split = dataset["validation"]
examples = validation_split.to_list()

for example in tqdm(examples):
    add_parse_to_example(example)

 39%|███▊      | 193/500 [04:04<06:28,  1.27s/it]


KeyboardInterrupt: 

In [ ]:
Dataset.from_list(train_docs)